# Random forest for pianist classification

## Import dependencies, set constants etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from joblib import Parallel, delayed, dump
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.inspection._permutation_importance import _weights_scorer, _create_importances_bunch
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import check_scoring
from sklearn.model_selection import StratifiedKFold, ParameterSampler, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import check_random_state
from statsmodels.stats.outliers_influence import variance_inflation_factor

import src.visualise.visualise_utils as vutils
from src import utils
from src.detect.detect_utils import OnsetMaker
from src.features.features_utils import *
from src.visualise.random_forest_plots import *

In [ ]:
# These variables are used for the optimization process
SEED = 42
N_FOLDS = 5
N_JOBS = -1
# Number of iterations to use in random sampling
N_ITER = 10000

In [ ]:
# Set the seed in NumPy for consistent results across function calls
np.random.seed(SEED)

## Define predictor and prediction variables, get mappings

In [ ]:
# Define target/prediction variables
EXOG_INS = 'piano'
PREDICTION = 'pianist'

In [ ]:
# These are the underlying categories each predictor belongs to
PREDICTORS_CATEGORIES = {
    'Swing': ['bur_log_mean', 'bur_log_std'],
    'Complexity': ['lz77_mean', 'lz77_std', 'n_onsets_mean', 'n_onsets_std'],
    'Feel': ['bass_prop_async_nanmean', 'drums_prop_async_nanmean', 'bass_prop_async_nanstd', 'drums_prop_async_nanstd'],
    'Interaction': ['self_coupling', 'coupling_drums', 'coupling_bass', 'coupling_piano_drums', 'coupling_piano_bass'],
    'Tempo': ['rolling_std_median', 'tempo', 'tempo_slope',]
}
PREDICTORS = [it for sl in list(PREDICTORS_CATEGORIES.values()) for it in sl]

In [ ]:
# This creates a dictionary for mapping a predictor onto its category
CATEGORY_MAPPING = {}
for pred in PREDICTORS:
    for category in PREDICTORS_CATEGORIES.keys():
        if pred in PREDICTORS_CATEGORIES[category]:
            CATEGORY_MAPPING[pred] = category

## Load in data

First, we load in our list of `src.detect.detect_utils.OnsetMaker` classes. These contain the location of detected onsets and beats, as well as additional metadata.

In [ ]:
onsets: list[OnsetMaker] = utils.unserialise_object(fr'{utils.get_project_root()}\models\matched_onsets_corpus_chronology')

## Extract features

Now, we can extract our desired feature from each OnsetMaker class.

In [ ]:
def get_feature_data(feature_cls, cols, extra_str = '', **cls_kwargs):
    """Creates a class with given kwargs and returns the desired key-value pairs from its summary dictionary"""
    cls = feature_cls(**cls_kwargs)
    return {k + extra_str: v for k, v in cls.summary_dict.items() if k in cols}

In [ ]:
def process_track(track: OnsetMaker) -> dict:
    """Processes a single track, extracting all required features, and returns a dictionary"""
    # Convert the summary dictionary (dictionary of arrays) to a dataframe
    summary_dict = pd.DataFrame(track.summary_dict)
    # These are the positions of downbeats, i.e. the first beat of a measure
    downbeats = track.ons['downbeats_manual']
    # The tempo and time signature of the track
    tempo = track.tempo
    time_signature = track.item['time_signature']
    # Subset to get my onsets and partner onsets as separate dataframes
    my_onsets = track.ons[EXOG_INS]
    my_beats = summary_dict[EXOG_INS]
    their_beats = summary_dict[[i for i in utils.INSTRUMENTS_TO_PERFORMER_ROLES.keys() if i != EXOG_INS]]
    # BEAT-UPBEAT RATIO
    bur = get_feature_data(
        BeatUpbeatRatio, ['bur_log_mean', 'bur_log_std', 'bur_log_count_nonzero'],
        my_onsets=my_onsets, my_beats=my_beats, clean_outliers=True
    )
    # PHASE CORRECTION
    pc = get_feature_data(
        PhaseCorrection, ['self_coupling', 'coupling_bass', 'coupling_drums', 'nobs'],
        my_beats=my_beats, their_beats=their_beats, order=1
    )
    # PHASE CORRECTION - PARTNER
    # In comparison to the 'full' phase correction model, we only need to get a few columns here
    pcb = get_feature_data(
        PhaseCorrection, ['coupling_piano', 'nobs'], extra_str='_bass',
        my_beats=summary_dict['bass'], their_beats=summary_dict[['piano', 'drums']], order=1
    )
    pcd = get_feature_data(
        PhaseCorrection, ['coupling_piano', 'nobs'], extra_str='_drums',
        my_beats=summary_dict['drums'], their_beats=summary_dict[['piano', 'bass']], order=1
    )
    # PROPORTIONAL ASYNCHRONY
    pa = get_feature_data(
        ProportionalAsynchrony, ['prop_async_count_nonzero', 'bass_prop_async_nanmean', 'drums_prop_async_nanmean', 'bass_prop_async_nanstd', 'drums_prop_async_nanstd'],
        summary_df=summary_dict, my_instr_name=EXOG_INS
    )
    # IOI COMPLEXITY
    ioi = get_feature_data(
        IOIComplexity, ['lz77_mean', 'lz77_std', 'n_onsets_mean', 'n_onsets_std', 'window_count', 'ioi_count'],
        my_onsets=my_onsets, downbeats=downbeats, tempo=tempo, time_signature=time_signature
    )
    # TEMPO SLOPE
    ts = get_feature_data(
        TempoSlope, ['tempo_slope', 'tempo_drift'],
        my_beats=pd.concat([my_beats, their_beats], axis=1).mean(axis=1)
    )
    # TEMPO STABILITY
    tstab = get_feature_data(
        RollingIOISummaryStats, ['rolling_std_count_nonzero', 'rolling_std_median'],
        my_onsets=my_beats, downbeats=downbeats, bar_period=4
    )
    # Return a single dictionary that combines the summary dictionary for all the features
    return dict(**track.item, **bur, **pc, **pcb, **pcd, **pa, **ioi, **ts, **tstab, tempo=tempo)

Now, we extract features from all tracks in parallel (should take < 5 minutes)

In [ ]:
with Parallel(n_jobs=-1, verbose=5) as parallel:
    res = parallel(delayed(process_track)(t) for t in onsets)

We can now convert all of these features to a dataframe, sort by the pianist in the recording, and print a few rows.

In [ ]:
df = pd.DataFrame(res).sort_values('pianist').reset_index(drop=True)

In [ ]:
print(df.head())

## Plot correlations between features
Before we clean outliers, we should plot the correlations between the raw values

In [ ]:
df[PREDICTORS].isna().sum()

In [ ]:
HeatMapFeatureCorrelation(df[PREDICTORS]).create_plot()
plt.show()

## Clean dataset
### Identify outliers
When cleaning the dataset, we first need to identify outlying values in our features. We set outlying values to `np.nan` so that we can set them to the overall average later, rather than removing the track completely. To start, we clean `self_coupling`, `coupling_bass`, or `coupling_drums` when `nobs < 30`: this means that we have fewer than 10 terms for each of our 3 predictors in the regression.

In [ ]:
clean = df.copy(deep=True)

In [ ]:
for col in ['self_coupling', 'coupling_bass', 'coupling_drums']:
    # If we have NaN obs, it's because the model failed to compile, so replace this with 0
    clean[f'nobs'] = clean[f'nobs'].fillna(0)
    clean.loc[clean['nobs'] < 30, col] = np.nan

For the same reason, we clean `coupling_bass_piano` (i.e. bass coupling to piano) when `nobs_bass < 30` (sim. for `coupling_drums_piano` and `nobs_drums`)

In [ ]:
for col in ['bass', 'drums']:
    # If we have NaN obs, it's because the model failed to compile, so replace this with 0
    clean[f'nobs_{col}'] = clean[f'nobs_{col}'].fillna(0)
    clean.loc[clean[f'nobs_{col}'] < 30, f'coupling_piano_{col}'] = np.nan

We clean `bur_log_mean` and `bur_log)std` when `-2 <= average_bur <= 2` (these are the values discussed by Corcoran and Frieler) OR `bur_log_count_nonzero < 15` (we have fewer than 15 BURs in the track).

In [ ]:
for col in ['bur_log_mean', 'bur_log_std']:
    clean.loc[clean['bur_log_mean'] < -2, col] = np.nan
    clean.loc[clean['bur_log_mean'] > 2, col] = np.nan
    clean.loc[clean['bur_log_count_nonzero'] < 15, col] = np.nan

We clean `bass/drums_prop_async` when `piano_prop_async_count_nonzero < 15`, i.e. we have fewer than 15 beat asynchronies

In [ ]:
for col in ['bass', 'drums']:
    clean.loc[clean['prop_async_count_nonzero'] < 10, f'{col}_prop_async_nanmean'] = np.nan
    clean.loc[clean['prop_async_count_nonzero'] < 10, f'{col}_prop_async_nanstd'] = np.nan

We clean `tempo_slope` and `tempo_drift` when `tempo_slope.abs() < 0.15`, i.e. we accelerated (or decelerated) by more than 0.15 beats-per-minute-per-second. This is the upper limit of values suggested to be realistic for jazz ensembles in the control condition of our previous experiment.

In [ ]:
clean.loc[clean['tempo_slope'].abs() > 0.15, 'tempo_slope'] = np.nan
clean.loc[clean['tempo_slope'].abs() > 0.15, 'tempo_drift'] = np.nan

We clean `rolling_std_median` when `rolling_std_median > 0.4`, i.e. the median standard deviation for a four-second window of 1/4 note beats is greater than 40 milliseconds. This is the upper limit of values for this variable from our earlier experiment.

In [ ]:
clean.loc[clean['rolling_std_median'] > 0.4, 'rolling_std_median'] = np.nan

## Get number of missing values per predictor
Let's see how many missing values we have per predictor variable

In [ ]:
COL_MAPPING = {
    'bur_log_mean': 'Beat-upbeat ratio, mean',
    'bur_log_std': 'Beat-upbeat ratio, std.',
    'lz77_mean': 'Compression score, mean',
    'lz77_std': 'Compression score, std',
    'n_onsets_mean': 'Onset density, mean',
    'n_onsets_std': 'Onset density, std',
    'bass_prop_async_nanmean': 'Piano→Bass, async mean',
    'bass_prop_async_nanstd': 'Piano→Bass, async std.',
    'drums_prop_async_nanmean': 'Piano→Drums, async mean',
    'drums_prop_async_nanstd': 'Piano→Drums, async std.',
    'coupling_bass': 'Piano→Bass, coupling',
    'coupling_piano_bass': 'Bass→Piano, coupling',
    'coupling_drums': 'Piano→Drums, coupling',
    'coupling_piano_drums': 'Drums→Piano, coupling',
    'self_coupling': 'Piano→Piano, coupling',
    'rolling_std_median': 'Tempo stability',
    'tempo': 'Tempo average',
    'tempo_slope': 'Tempo slope',
}

In [ ]:
miss = clean[PREDICTORS].isna().sum().reset_index()
miss['cat'] = miss['index'].map(CATEGORY_MAPPING)
print(miss)

In [ ]:
print(miss.groupby('cat')[0].mean() / 300)

In [ ]:
clean.head(10)

In [ ]:
CountPlotMissingValues(clean[PREDICTORS], CATEGORY_MAPPING).create_plot()
plt.show()

### Impute missing values based on feature average
We can now set missing values to the average for that feature across the entire dataset.

In [ ]:
X = clean[PREDICTORS].fillna(clean[PREDICTORS].mean())

In [ ]:
X.corr()

Let's check that we've filled all missing values

In [ ]:
print(X.isna().values.any())

### Encode categorical prediction variable
Next, we need to encode our predictor variable (currently a list of names) to integers.

In [ ]:
# We can store the labels for later use
pianist_labels = clean[PREDICTION].unique().tolist()
y = clean[PREDICTION].map({val: i for i, val in enumerate(pianist_labels)})

## Random Forest modelling (new)
### Create the model class

In [ ]:
PREDICTORS_CATEGORIES

In [ ]:
class RandomForestFit:
    """Provides a wrapper around `StratifiedKFold` and `RandomForestClassifier`"""
    col_idxs = [[0, 1], [2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17]]

    def __init__(self, params: dict = None):
        self.params = params if params is not None else {}
        # I think we can probably shuffle the data first.
        # The data is unordered (not time-series), so unlikely to have a negative effect
        self.skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
        self.predicts, self.importances, self.grouped_importances = [], [], []
        self.trees = []

    def fit_model(self, x_data, y_data):
        fits = [self._fit_model(x_data, y_data, x_idx, y_idx) for x_idx, y_idx in self.skf.split(x_data, y_data)]
        predicts, feature_importances, grouped_importance = list(zip(*fits))
        self.predicts = self._format_predicts_df(predicts)
        self.importances = self._format_importances_df(feature_importances)
        self.grouped_importances = pd.concat(grouped_importance, axis=1)
        return self

    def get_global_accuracy(self) -> float:
        """Returns the percentage of correct predictions"""
        return self.predicts['correct'].value_counts(normalize=True)[True]

    @staticmethod
    def _format_importances_df(importances) -> pd.DataFrame:
        importances_df_ = (
            pd.concat(importances, axis=1)
            .reset_index(drop=False)
            .rename(columns={'index': 'feature'})
        )
        importances_df_['category'] = importances_df_['feature'].map(CATEGORY_MAPPING)
        return importances_df_

    @staticmethod
    def _format_predicts_df(predicts) -> pd.DataFrame:
        predict_df_ = (
            pd.concat(predicts, axis=0)
            .sort_index()
            .rename(columns={0: 'prediction', 'pianist': 'actual'})
        )
        predict_df_['correct'] = predict_df_['prediction'] == predict_df_['actual']
        for var in ['prediction', 'actual']:
            predict_df_[var] = predict_df_[var].map({i: val for i, val in enumerate(pianist_labels)})
        return predict_df_

    def _fit_model(self, x_data, y_data, train_idx, test_idx):
        # Get our splits for this fold
        X_train, X_test = x_data.iloc[train_idx], x_data.iloc[test_idx]
        y_train, y_test = y_data[train_idx], y_data[test_idx]
        # Create the random forest model
        rf = RandomForestClassifier(random_state=SEED, **self.params)
        # Fit the random forest to the training data
        rf.fit(X_train, y_train)
        self.trees.append(rf)
        # Predict the test data
        y_predict = rf.predict(X_test)
        # Get prediction probabilities
        y_probs = rf.predict_proba(X_test)
        # Calculate importance of individual features
        fpi = self.permutation_importance(rf, X_test, y_test)
        # Calculate importance of feature groups
        gpi = self.grouped_permutation_importance(rf, X_test, y_test)
        # Return the prediction and feature importance dataframes
        return (
            self._format_predict_df(y_predict, y_probs, y_test),
            pd.DataFrame(fpi.importances, index=X.columns),
            pd.DataFrame(gpi.importances, index=PREDICTORS_CATEGORIES.keys())
        )

    @staticmethod
    def permutation_importance(
            estimator, X: pd.DataFrame, y: pd.DataFrame, n_repeats: int = 10,
            scoring: str = 'accuracy', sample_weight: float = None, max_samples: float = 1.0
    ) -> dict:
        return permutation_importance(
            estimator, X, y, n_repeats=n_repeats, random_state=SEED, n_jobs=N_JOBS,
            sample_weight=sample_weight, max_samples=max_samples, scoring=scoring
        )

    @staticmethod
    def _format_predict_df(predict, predict_probs, actual):
        y_probs = pd.DataFrame(predict_probs)
        y_probs.columns = y_probs.columns.map({i: val for i, val in enumerate(pianist_labels)})
        y_probs.index = actual.index
        # Format the prediction series
        y_predict_s = pd.Series(predict)
        y_predict_s.index = actual.index
        # Concatenate the predicted and actual pianist
        return pd.concat([y_predict_s, actual, y_probs], axis=1)

    @staticmethod
    def _grouped_permutation_score(
        estimator, X, y, sample_weight, col_idxs,
        random_state, n_repeats, scorer,
    ):
        random_state = check_random_state(random_state)
        X_permuted = X.copy()
        scores = []
        shuffling_idx = np.arange(X_permuted.shape[0])
        for _ in range(n_repeats):
            random_state.shuffle(shuffling_idx)
            col = X_permuted.iloc[shuffling_idx, col_idxs]
            col.index = X_permuted.index
            X_permuted[X_permuted.columns[col_idxs]] = col
            scores.append(_weights_scorer(scorer, estimator, X_permuted, y, sample_weight))
        return np.array(scores)

    def grouped_permutation_importance(
            self, estimator, X: pd.DataFrame, y: pd.DataFrame, random_state: float = SEED,
            n_repeats: int = 10, scoring: str = 'accuracy', sample_weight: float = None
    ) -> dict:
        random_state = check_random_state(random_state)
        random_seed = random_state.randint(np.iinfo(np.int32).max + 1)
        scorer = check_scoring(estimator, scoring)
        baseline_score = _weights_scorer(scorer, estimator, X, y, sample_weight)
        scores2 = [self._grouped_permutation_score(
            estimator=estimator, X=X, y=y, col_idxs=col_idx, random_state=random_seed,
            n_repeats=n_repeats, scorer=scorer, sample_weight=sample_weight,
        ) for col_idx in self.col_idxs]
        return _create_importances_bunch(baseline_score, np.array(scores2))


### Get baseline accuracy
Here, we just guess a random pianist for each track

In [ ]:
accuracy_random = accuracy_score(y, np.random.randint(y.min(), y.max(), len(y)))
print(f"Random guess accuracy: {accuracy_random}")

As you'd expect with 10 pianists and 30 tracks per pianist, we'll be correct approximately one in ten times if we just guess the pianist at random.

### Fit the initial model and get accuracy
Here we just use the default parameters for fitting the model

In [ ]:
rf_init = RandomForestFit().fit_model(X, y)
print(f'Initial model accuracy: {rf_init.get_global_accuracy()}')

### Parameter optimization
Now, we can try and optimize the model using a randomized search over an array of parameters

In [ ]:
# These are the parameters we'll sample from when optimizing
test_params = dict(
    # The number of trees to grow in the forest
    n_estimators=[i for i in range(10, 1001, 1)],
    # Max number of features considered for splitting a node
    max_features=[None, 'sqrt', 'log2'],
    # Max number of levels in each tree
    max_depth=[None, *[i for i in range(1, 101, 1)]],
    # Minimum number of samples required to split a node
    min_samples_split=[i for i in range(2, 11)],
    # Minimum number of samples required at each leaf node
    min_samples_leaf=[i for i in range(1, 11)],
    # Whether to sample data points with our without replacement
    bootstrap=[True, False],
)

In [ ]:
# Create the `ParameterSampler` object for the required number of iterations
sampler = ParameterSampler(test_params, n_iter=N_ITER, random_state=SEED)

In [ ]:
def get_cached_tracks() -> list:
    """Tries to get cached optimised parameters, returns an empty list if cannot find"""
    try:
        return utils.load_csv(
            fpath=rf"{utils.get_project_root()}\references\parameter_optimisation\corpus_chronology",
            fname='random_forest_opt'
        )
    except FileNotFoundError:
        return []

In [ ]:
def create_forest(parameters: dict, iteration: int,):
    """Conducts one iteration of parameter optimisation, with given parameter"""
    # Check our cache to see if we've already got results for this iteration
    # As we use a set seed with `ParameterSampler`, we can get results with just the counter
    cached_results = get_cached_tracks()
    try:
        cached_res = [o for o in cached_results if o['iteration'] == iteration][0]
        # assert all(k in cached_res and parameters[k] == cached_res[k] for k in to_check)
    except (IndexError, AssertionError):
        pass
    else:
        return cached_res

    # Create the forest model
    forest = RandomForestFit(params=parameters)
    # Fit the model to the data
    forest.fit_model(X, y)
    # Create the results dictionary and save
    results_dict = {
        'accuracy': forest.get_global_accuracy(),
        'iteration': iteration,
        **parameters
    }
    utils.save_csv(
        results_dict,
        rf"{utils.get_project_root()}\references\parameter_optimisation\corpus_chronology",
        'random_forest_opt'
    )
    # Return the fitted forest object
    return results_dict

In [ ]:
# Use lazy parallelization to create the forest and fit to the data
with Parallel(n_jobs=N_JOBS, verbose=11) as parallel:
    rfs_fitted = parallel(delayed(create_forest)(params, num) for num, params in enumerate(sampler))

### Get the optimized parameters

In [ ]:
# Create a dataframe of parameters and accuracy scores
opt_df = pd.DataFrame(rfs_fitted)
opt_df.head()

In [ ]:
# Get the parameter combination that yielded the best accuracy
best_params = opt_df[opt_df['accuracy'] == opt_df['accuracy'].max()][test_params.keys()].to_dict('records')[0]
best_params['max_depth'] = int(best_params['max_depth'])

So the optimization process doesn't have to be re-run each time, the final parameter set is given below in plain text:

`{
    'n_estimators': 50,
    'max_features': 'log2',
    'max_depth': 30,
    'bootstrap': True,
    'min_samples_leaf': 1,
    'min_samples_split': 2
 }`

### Fit the optimized model and get accuracy
Now, we use the best combination of parameters from our optimization process

In [ ]:
BEST_PARAMS = {'n_estimators': 50, 'max_features': 'log2', 'max_depth': 30, 'bootstrap': True, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [ ]:
rf_opt = RandomForestFit(params=BEST_PARAMS).fit_model(X, y)
print(f'Optimized model accuracy: {rf_opt.get_global_accuracy()}')

## Serialise model and save

In [ ]:
rf_all = RandomForestClassifier(random_state=SEED, **BEST_PARAMS)
rf_all.fit(X, clean[PREDICTION])
dump(rf_all, f'{utils.get_project_root()}/models/pianist_predictor.joblib')

## Plot results
### Plot feature importance
Which rhythmic features are most important in defining the personal style of a particular pianist?

In [ ]:
HeatMapPredictionProbDendro(rf_opt.predicts, include_images=False).create_plot()
plt.show()

### Get accuracy scores
Mean accuracy, top-k (`k=3`) accuracy, Cohen's kappa

In [ ]:
from sklearn.metrics import top_k_accuracy_score, accuracy_score, cohen_kappa_score, confusion_matrix
act = rf_opt.predicts['actual'].map({v: k for k, v in enumerate(rf_opt.predicts['actual'].unique())}).values
pred = rf_opt.predicts['prediction'].map({v: k for k, v in enumerate(rf_opt.predicts['actual'].unique())}).values
probs = rf_opt.predicts[pianist_labels].values
print('Mean accuracy:', accuracy_score(act, pred))
print('Top-k accuracy:', top_k_accuracy_score(act, probs, k=3))
print('Cohen kappa:', cohen_kappa_score(act, pred))

In [ ]:
cm = pd.DataFrame(confusion_matrix(act, pred, normalize='true'))
cm.index = pianist_labels
cm.columns = pianist_labels
cm

### Get feature importances
Average importance per feature, per category

In [ ]:
imp_feat = (
    rf_opt.importances.copy(deep=True)
    .melt(id_vars=['feature', 'category'])
    .groupby('feature')
    ['value']
    .mean()
    .sort_values()
)
print(imp_feat * 100)

In [ ]:
imp_cat = (
    rf_opt.grouped_importances.copy(deep=True)
    .mean(axis=1)
    .sort_values(ascending=True)
)
print(imp_cat * 100)

## Predicting first and last recordings

In [ ]:
X = clean[PREDICTORS].fillna(clean[PREDICTORS].mean())
# We can store the labels for later use
pianist_labels = clean[PREDICTION].unique().tolist()
y = clean[PREDICTION].map({val: i for i, val in enumerate(pianist_labels)})

In [ ]:
clean['recording_year_'] = clean['recording_year'].astype(int)

In [ ]:
min_idxs = (
    clean.sort_values(by=['pianist', 'recording_year_', 'mbz_id'], ascending=True)
    .groupby('pianist')
    .head(6)
    .index
    .sort_values()
    .to_numpy()
)
max_idxs = (
    clean.sort_values(by=['pianist', 'recording_year_', 'mbz_id'], ascending=False)
    .groupby('pianist')
    .head(6)
    .index
    .sort_values()
    .to_numpy()
)

In [ ]:
def minmax_acc(test_idx) -> float:
    train_idx = np.sort(np.array([*(set(X.index) - set(test_idx))]))
    assert not set(test_idx).issubset(set(train_idx))
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    # Create the random forest model
    rf_fl = RandomForestClassifier(random_state=SEED, **BEST_PARAMS)
    # Fit the random forest to the training data
    rf_fl.fit(X_train, y_train)
    fl_pred = rf_fl.predict(X_test)
    # Get accuracy for predicting the set
    return accuracy_score(y_test, fl_pred)

min_acc = minmax_acc(min_idxs)
max_acc = minmax_acc(max_idxs)
print(min_acc, max_acc)

In [ ]:
def permute_model(rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=rand_state)
    # Create the random forest model
    rf_rand = RandomForestClassifier(random_state=SEED, **BEST_PARAMS)
    # Fit the random forest to the training data
    rf_rand.fit(X_train, y_train)
    rand_pred = rf_rand.predict(X_test)
    return accuracy_score(y_test, rand_pred)

In [ ]:
with Parallel(n_jobs=-1, verbose=5) as parallel:
    accs = parallel(delayed(permute_model)(state) for state in range(vutils.N_BOOT))

In [ ]:
np.std(accs)

In [ ]:
HistPlotFirstLastP(accs, min_acc, max_acc).create_plot()
plt.show()

## Regressions of age vs strongest predictors

In [ ]:
clean['recording_year_'] = clean['recording_year'].astype(int)
clean['career_progress'] = clean.groupby('pianist')['recording_year_'].transform(lambda g: g - g.min())
clean['career_progress'] /= clean['career_progress'].max()
clean['jazz_progress'] = clean['recording_year_'] - clean['recording_year_'].min()
clean['jazz_progress'] /= clean['jazz_progress'].max()

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

predictors = ['drums_prop_async_nanmean', 'tempo_slope', 'bur_log_mean', 'n_onsets_mean', 'coupling_piano_drums']
conr = []
marr = []
for predict in predictors:
    vars_ = [predict, 'pianist', 'career_progress', 'jazz_progress']
    data = clean.copy(deep=True)
    data[predict] = data[predict].fillna(data[predict].mean())
    md = smf.mixedlm(f'{predict}~career_progress', groups=data['pianist'], data=data, re_formula='~career_progress').fit(method=['lbfgs'])
    print(md.summary())
    # Variance explained by the fixed effects: we need to use md.predict() with the underlying data to get this
    var_fixed = md.predict().var()
    # Variance explained by the random effects
    var_random = float(md.cov_re.to_numpy().mean())
    # Variance of the residuals
    var_resid = md.scale
    # Total variance of the model
    total_var = var_fixed + var_random + var_resid
    # Calculate the r2 values and append to the model
    conr.append((var_fixed + var_random) / total_var)
    marr.append(var_fixed / total_var)
print('mean conditional r2', np.mean(conr) * 100, 'stdev', np.std(conr) * 100)
print('mean marginal r2', np.mean(marr) * 100, 'stdev', np.std(marr) * 100)

In [ ]:
clean.groupby('pianist')['recording_year_'].describe()

In [ ]:
import seaborn as sns
class RegPlotCareerJazzProgress(vutils.BasePlot):
    predictors = ['drums_prop_async_nanmean', 'tempo_slope', 'bur_log_mean', 'n_onsets_mean', 'coupling_piano_drums']
    palette = sns.color_palette('tab10')
    palette = [palette[2], palette[4], palette[0], palette[1],   palette[3]]
    markers = ['o', 's', 'D', '^', 'p']
    categories = ['Feel', 'Tempo', 'Swing', 'Complexity', 'Interaction']
    REG_KWS = dict(color=vutils.BLACK, linewidth=vutils.LINEWIDTH * 2, ls=vutils.LINESTYLE)
    xspace = np.linspace(0, 1, 100)
    N_JOBS = -1

    def __init__(self, model_df, cat_mapping, **kwargs):
        self.corpus_title = 'corpus_chronology'
        self.cat_mapping = cat_mapping
        self.df = model_df.copy(deep=True)
        super().__init__(
            figure_title=fr'random_forest_plots\regplot_careerjazzprogress_{self.corpus_title}', **kwargs
        )
        self.fig, self.ax = plt.subplots(
            nrows=2, ncols=3, figsize=(vutils.WIDTH, vutils.WIDTH / 2), sharex=True, sharey=False
        )
        self.ax.flatten()[-1].axis('off')

    def get_line(self, model, mean):
        params = model.params
        intercept = params['Intercept']
        jp = params['jazz_progress'] * mean
        return [intercept + jp + (i * params['career_progress']) for i in self.xspace]

    @staticmethod
    def fit_model(data_, predict_):
        return (
            smf.mixedlm(
                f'{predict_}~career_progress+jazz_progress',
                groups=data_['pianist'],
                data=data_,
                re_formula='~jazz_progress'
            ).fit(method=['lbfgs'])
        )

    def bootstrap(self, data_, predict_) -> pd.DataFrame:
        def shuffle_data_and_fit(state):
            samp = data_.sample(frac=1, replace=True, random_state=state)
            mode = self.fit_model(samp, predict_)
            return pd.Series(self.get_line(mode, np.mean(samp['jazz_progress'])))

        with Parallel(n_jobs=self.N_JOBS, verbose=3) as par:
            boots = par(delayed(shuffle_data_and_fit)(num) for num in range(vutils.N_BOOT))
        boot_ys = pd.concat(boots, axis=1)
        low = boot_ys.apply(lambda r: np.percentile(r, 2.5), axis=1)
        high = boot_ys.apply(lambda r: np.percentile(r, 97.5), axis=1)
        return pd.DataFrame(dict(x=self.xspace, low=low, high=high))

    def _create_plot(self):
        for ax, predict, col, mark in zip(self.ax.flatten(), self.predictors, self.palette, self.markers):
            data = self.df.copy(deep=True)
            data[predict] = data[predict].fillna(data[predict].mean())
            md = self.fit_model(data, predict)
            sns.scatterplot(
                data=data, x='career_progress', y=predict, label=self.cat_mapping[predict],
                ax=ax, color=col, marker=mark, s=100, alpha=vutils.ALPHA
            )
            results_y = pd.DataFrame(self.get_line(md, np.mean(data['jazz_progress'])))
            ax.plot(self.xspace, results_y, **self.REG_KWS)
            boot_df = self.bootstrap(data, predict)
            ax.fill_between(boot_df['x'], boot_df['low'], boot_df['high'], color=vutils.BLACK, alpha=vutils.ALPHA)

    def _format_ax(self):
        leg, hand = [], []
        for a, tit, col in zip(self.ax.flatten(), self.categories, self.palette):
            l, h = a.get_legend_handles_labels()
            leg.extend(l)
            hand.extend(h)
            a.get_legend().remove()
            a.grid(axis='x', which='major', **vutils.GRID_KWS)
            a.set_title(tit, color=col)
            a.set_ylabel(COL_MAPPING[a.get_ylabel()], color=col)
            a.set(xlabel='', xlim=(-0.05, 1.05))
            plt.setp(a.spines.values(), linewidth=vutils.LINEWIDTH)
            a.tick_params(axis='both', bottom=True, width=vutils.TICKWIDTH)
        self.fig.legend(
            leg, hand, loc='lower right', title='Category', frameon=True,
            framealpha=1, edgecolor=vutils.BLACK, bbox_to_anchor=(0.9, 0.15)
        )

    def _format_fig(self):
        self.fig.supxlabel('Career progress')
        self.fig.subplots_adjust(left=0.07, right=0.99, bottom=0.1, top=0.95, wspace=0.25)

In [ ]:
RegPlotCareerJazzProgress(clean, CATEGORY_MAPPING).create_plot()
plt.show()

## Binary logistic regression of two clusters -- impressionistic vs blues

In [ ]:
impression = ['Bill Evans', 'Bud Powell', 'John Hicks', 'Keith Jarrett', 'Tommy Flanagan']
bop = ['Ahmad Jamal', 'Junior Mance', 'Kenny Barron', 'McCoy Tyner', 'Oscar Peterson']

In [ ]:
clean.loc[clean['pianist'].isin(impression), 'cluster'] = 0
clean.loc[clean['pianist'].isin(bop), 'cluster'] = 1

In [ ]:
X = clean[reversed([
    'bass_prop_async_nanmean',
    'bass_prop_async_nanstd',
    'drums_prop_async_nanmean',
    'drums_prop_async_nanstd',
    'tempo',
    'tempo_slope',
    'rolling_std_median',
    'bur_log_mean',
    'bur_log_std',
    'n_onsets_mean',
    'n_onsets_std',
    'lz77_mean',
    'lz77_std',
    'self_coupling',
    'coupling_bass',
    'coupling_drums',
    'coupling_piano_bass',
    'coupling_piano_drums'
])]
X = X.fillna(X.mean()).apply(stats.zscore)
X = sm.add_constant(X)
y = clean['cluster'].astype(int)

In [ ]:
pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

In [ ]:
md = sm.Logit(y, X).fit()
print(md.summary())

In [ ]:
y_pred = md.predict(X)
print(accuracy_score(y, y_pred.round().astype(int)))
print(roc_auc_score(y, y_pred))
print(roc_auc_score(y, np.random.choice([0, 1], 300)))

In [ ]:
RocPlotLogRegression(y, y_pred).create_plot()
plt.show()

In [ ]:
params = pd.concat([md.params.rename('coeff'), md.conf_int().rename(columns={0: 'low', 1: 'high'})], axis=1).apply(np.exp).reset_index(drop=False)[1:]
# params['low'] = params['coeff'] - params['low']
# params['high'] -= params['coeff']
params['category'] = params['index'].map(CATEGORY_MAPPING)

In [ ]:
params

In [ ]:
StripPlotLogitCoeffs(md, category_mapping=CATEGORY_MAPPING).create_plot()
plt.show()